In [ ]:
# Census race and SVI data 
*Title*:Updates to Race Profile of LA County Residents
*Author*:Divine Mutoni 
*Description*: This notebook focuses on the race variables in the ACS 2022 5-year estimates data, the Social Vulnerability Index data, and data on neighboorhood boundaries in LA city. 

In [ ]:
## Load, and clean race census tract data and SVI
import pandas as pd
import geopandas as gpd

cens_race = pd.read_csv('Data/ACS 2022_Race_5YEst.csv', #import censu tract
                     dtype={'FIPS':str,}
                     )
cens_race.columns

In [ ]:
cens_geo = gpd.read_file('Data/Census_Tracts_2020_DM.geojson')
cens_geo = cens_geo[['CT20','geometry']]
cens_geo['FIPS'] = '06'+ '037' + cens_geo['CT20']
cens_race_geo= cens_geo.merge(cens_race,on="FIPS")

In [ ]:
svi = pd.read_csv('Data/SVI California 2020.csv',
                  dtype={'FIPS':str,}
                 )

In [ ]:
svi_race = cens_race_geo.merge(svi,on='FIPS')
svi_race.columns

In [ ]:
## Analysis of SVI score across racial demographics in LA county
### Prompt 1: What is the distribution of SVI scores (mean) across different racial/ethnic majority census tracts?

In [ ]:
svi_race['PCT_Black'] = svi_race['Total Population: Not Hispanic or Latino: Black or African American Alone']/svi_race['Total Population:']*100
svi_race['PCT_White'] = svi_race['Total Population: Not Hispanic or Latino: White Alone']/svi_race['Total Population:']*100
svi_race['PCT_Hispanic'] = svi_race['Total Population: Hispanic or Latino']/svi_race['Total Population:']*100
svi_race['PCT_Asian'] = svi_race['Total Population: Not Hispanic or Latino: Asian Alone']/svi_race['Total Population:']*100

In [ ]:
# Create data frames of four ethnic groups and show the percentages (SPL_THEME variable which captures the social vulnerability scores across 4 metrics
![alt text](SVI_Score.png "SVI_Score")

In [ ]:
df_Hmaj = svi_race[svi_race['PCT_Hispanic']>50]
df_NHWhite_maj = svi_race[svi_race['PCT_White']>50]
df_NHBlack_maj = svi_race[svi_race['PCT_Black']>50]
df_NHAsian_maj = svi_race[svi_race['PCT_Asian']>50]

In [ ]:
# Import analysis and plotting modules 
from pandas.plotting import bootstrap_plot
import numpy as np
import folium
import json
import matplotlib.pyplot as plt
import mapclassify
from mapclassify import NaturalBreaks

In [ ]:
# 2.1. Plot Hispanic_majority
data = df_Hmaj['SPL_THEMES'].dropna().values

# Function to generate bootstrap samples
def generate_bootstrap_samples(data, num_samples=1000):
    sample_means = np.zeros(num_samples)
    for i in range(num_samples):
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
        sample_means[i] = np.mean(bootstrap_sample)
    return sample_means

# Generate bootstrap samples
bootstrap_samples = generate_bootstrap_samples(data)

# Create the bootstrap plot
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_samples, bins=30, edgecolor='black', alpha=0.7,color='skyblue')
plt.title('Bootstrap Plot in Los Angeles')
plt.xlabel('Mean Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()
plt.savefig('SVI Scores_Hispmaj.png')

In [ ]:
# 2.2. Plot White_majority
data2 = df_NHWhite_maj['SPL_THEMES'].dropna().values

# Function to generate bootstrap samples
def generate_bootstrap_samples(data2, num_samples=1000):
    sample_means = np.zeros(num_samples)
    for i in range(num_samples):
        bootstrap_sample = np.random.choice(data2, size=len(data2), replace=True)
        sample_means[i] = np.mean(bootstrap_sample)
    return sample_means

# Generate bootstrap samples
bootstrap_samples = generate_bootstrap_samples(data2)

# Create the bootstrap plot
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_samples, bins=30, edgecolor='black', alpha=0.7,color='blue')
plt.title('Bootstrap Plot in Los Angeles')
plt.xlabel('Mean Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()
plt.savefig('SVI Scores_Whitemaj.png')

In [ ]:
# 2.3. Plot Black majority Census Tracts
data3 = df_NHBlack_maj['SPL_THEMES'].dropna().values

# Function to generate bootstrap samples
def generate_bootstrap_samples(data3, num_samples=1000):
    sample_means = np.zeros(num_samples)
    for i in range(num_samples):
        bootstrap_sample = np.random.choice(data3, size=len(data3), replace=True)
        sample_means[i] = np.mean(bootstrap_sample)
    return sample_means

# Generate bootstrap samples
bootstrap_samples = generate_bootstrap_samples(data3)

# Create the bootstrap plot
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_samples, bins=30, edgecolor='black', alpha=0.7,color='darkblue')
plt.title('SVI Scores_Asian maj')
plt.xlabel('Mean Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()
plt.savefig('SVI Scores_Blackmaj.png')

In [ ]:
# 2.4. Plot Asian_majority
data4 = df_NHAsian_maj['SPL_THEMES'].dropna().values

# Function to generate bootstrap samples
def generate_bootstrap_samples(data4, num_samples=1000):
    sample_means = np.zeros(num_samples)
    for i in range(num_samples):
        bootstrap_sample = np.random.choice(data4, size=len(data4), replace=True)
        sample_means[i] = np.mean(bootstrap_sample)
    return sample_means

# Generate bootstrap samples
bootstrap_samples = generate_bootstrap_samples(data4)

# Create the bootstrap plot
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_samples, bins=30, edgecolor='black', alpha=0.7,color='grey')
plt.title('SVI Scores_Asian maj')
plt.xlabel('Mean Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()
plt.savefig('SVI Scores_Asianmaj.png')

In [ ]:
### Prompt 2: Create an interactive map that plots the SVI scores (SPL_THEME) across maj. White or maj. minority column (Y/N)

In [ ]:
# 3.1. First create a binary column 'Majority' based on PCT_White
svi_race['Category'] = np.where(svi_race['PCT_White'] >= 50, 1, 0)
svi_race['Category'].head

In [ ]:
# 3.2. Plot interactive choloropeth map
data = svi_race.loc[svi_race['RPL_THEMES'] >= 0]

m = folium.Map(location=[34.2,-118.2], #create empty CartoDB base map
               tiles='CartoDB positron', 
               attribution='CartoDB')
#create choropleth
folium.Choropleth(
                  geo_data=data,
                  data=data,          
                  key_on='feature.properties.FIPS',
                  columns=['FIPS','RPL_THEMES'], 
                  fill_color='RdPu',
                  line_weight=0.1, 
                  fill_opacity=0.9,
                  line_opacity=0.9, 
                  legend_name='SVI Percentile ranking - LA Census Tracts').add_to(m)
m

In [ ]:
# 3.3. add the white and non-white binary to the feature group
gdf = gpd.GeoDataFrame(data)
for index, column in data.gdf.iteritems():
    color = 'yellow' if row['Category'] == 1 else 'grey'
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['LocationName'], icon=folium.Icon(color=color)).add_to(m)

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(m)
m

m.save('SVI_Percentiles.png')